In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from h2o.automl import H2OAutoML

In [10]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 22 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,"7 days, 23 hours and 33 minutes"
H2O_cluster_name:,H2O_from_python_deepakagrawal_of8gfg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.533 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [12]:
historical_data = h2o.import_file('/Users/deepakagrawal/AI Hackathon/ipl_train_data.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
h2o_df = h2o.H2OFrame(historical_data)

In [14]:
h2o_df.types


{'team1': 'enum',
 'team2': 'enum',
 'days': 'enum',
 'sessions': 'enum',
 'year': 'int',
 'location': 'enum',
 'popularity_team1': 'int',
 'popularity_team2': 'int',
 'pods_required': 'int'}

In [18]:
h2o_df.head()


team1,team2,days,sessions,year,location,popularity_team1,popularity_team2,pods_required
DC,PBKS,Thursday,Evening,2023,Kolkata,6,5,26
CSK,RCB,Friday,Afternoon,2024,Delhi,9,8,39
CSK,RCB,Tuesday,Evening,2022,Delhi,9,8,39
MI,LSG,Friday,Afternoon,2022,Mumbai,10,7,35
GT,MI,Thursday,Evening,2022,Delhi,8,10,41
LSG,MI,Monday,Evening,2021,Bangalore,7,10,37
PBKS,RR,Friday,Afternoon,2022,Jaipur,5,6,23
GT,CSK,Tuesday,Afternoon,2022,Lucknow,8,9,35
LSG,GT,Friday,Evening,2022,Hyderabad,7,8,34
LSG,PBKS,Thursday,Afternoon,2024,Delhi,7,5,27


In [19]:
pods_train, pods_test = h2o_df.split_frame(ratios = [0.8], seed = 42)

In [20]:
Y = "pods_required"
X = h2o_df.columns
X.remove(Y)

In [21]:
X

['team1',
 'team2',
 'days',
 'sessions',
 'year',
 'location',
 'popularity_team1',
 'popularity_team2']

In [22]:
Y

'pods_required'

In [23]:
# Train AutoML Model
aml = H2OAutoML(max_models=10, seed=42, exclude_algos=["StackedEnsemble", "DeepLearning", "GLM"], verbosity = "info")
aml.train(x = X, y = Y, training_frame = pods_train)

AutoML progress: |
15:34:17.494: Project: AutoML_2_20250404_153417
15:34:17.494: 5-fold cross-validation will be used.
15:34:17.494: Setting stopping tolerance adaptively based on the training frame: 0.05
15:34:17.494: Build control seed: 42
15:34:17.495: training frame: Frame key: AutoML_2_20250404_153417_training_py_5_sid_896b    cols: 9    rows: 195  chunks: 1    size: 7681  checksum: 51013270510065
15:34:17.495: validation frame: NULL
15:34:17.495: leaderboard frame: NULL
15:34:17.495: blending frame: NULL
15:34:17.495: response column: pods_required
15:34:17.495: fold column: null
15:34:17.495: weights column: null
15:34:17.495: AutoML: XGBoost is not available; skipping it.
15:34:17.495: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (7g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_2_20250404_153417


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    97                 97                          20056                  4            8            5.63918       10            14            11.8144

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.08272248237251635
RMSE: 0.28761516366929674
MAE: 0.2016550797682542
RMSLE: 0.007554125422958733
Mean Residual Deviance: 0.08272248237251635

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 2.0208046970592193
RMSE: 1.4215501036049414
MAE: 1.0133467773521372
RMSLE: 0.03907390034950462
Mean Residual Deviance: 2.0208046970592193

Cross-Validation Metrics Summary: 
                        mean       sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ---------  ----------  ------------  ------------  ------------  ------------  ------------
aic                     nan        0           nan           nan           nan           nan           nan
loglikelihood           nan        0           nan           nan           nan           nan           nan
mae                     1.01335    0.148194    0.839963      0.906888      1.00241       1.1137        1.20377
mean_residual_deviance  2.0208     0.549805    1.61221       1.58519       1.93379       2.03293       2.93989
mse                     2.0208     0.549805    1.61221       1.58519       1.93379       2.03293       2.93989
r2                      0.929257   0.0110409   0.937358      0.934253      0.934858      0.91014       0.929675
residual_deviance       2.0208     0.549805    1.61221       1.58519       1.93379       2.03293       2.93989
rmse                    1.41196    0.184294    1.26973       1.25904       1.39061       1.42581       1.71461
rmsle                   0.0386912  0.00609949  0.0333143     0.0357043     0.0350824     0.041052      0.0483029

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse        training_mae         training_deviance
---  -------------------  ----------  -----------------  -------------------  -------------------  -------------------
     2025-04-04 15:34:18  0.093 sec   0.0                5.393609094697508    4.376752136752138    29.09101906640367
     2025-04-04 15:34:18  0.096 sec   5.0                3.818775472115155    3.077710674970578    14.583046106428325
     2025-04-04 15:34:18  0.099 sec   10.0               2.8374332580161505   2.2732345189803684   8.051027493696147
     2025-04-04 15:34:18  0.102 sec   15.0               2.137454514955596    1.6927573668651092   4.568711803504061
     2025-04-04 15:34:18  0.105 sec   20.0               1.632368356869686    1.2646683032696064   2.6646264525094385
     2025-04-04 15:34:18  0.108 sec   25.0               1.30199385923277     0.9759893270639273   1.6951880094798422
     2025-04-04 15:34:18  0.111 sec   30.0               1.0581301851524993   0.7767110090989333   1.1196394887308625
     2025-04-04 15:34:18  0.114 sec   35.0               0.893689670295217    0.6454840635642027   0.7986812267923736
     2025-04-04 15:34:18  0.117 sec   40.0               0.7520195849980074   0.5424377637031751   0.5655334562205752
     2025-04-04 15:34:18  0.120 sec   45.0               0.652509291868691    0.4667068579258063   0.4257683759749806
---  ---                  ---         ---                ---                  ---                  ---
     2025-04-04 15:34:18  0.127 sec   55.0               0.5289479041490716   0.3804221617869842   0.2797858853036954
     2025-04-04 15:34:18  0.130 sec   60.0               0.47

In [24]:
lb = aml.leaderboard

In [25]:
lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_4_AutoML_2_20250404_153417,1.42155,2.0208,1.01335,0.0390739,2.0208
GBM_2_AutoML_2_20250404_153417,1.45432,2.11505,1.05492,0.0394216,2.11505
GBM_grid_1_AutoML_2_20250404_153417_model_2,1.45633,2.1209,1.10194,0.0398793,2.1209
GBM_grid_1_AutoML_2_20250404_153417_model_4,1.49023,2.22078,1.11085,0.0403211,2.22078
GBM_3_AutoML_2_20250404_153417,1.50678,2.2704,1.11146,0.0415382,2.2704
GBM_grid_1_AutoML_2_20250404_153417_model_1,1.99198,3.968,1.49541,0.0551912,3.968
GBM_grid_1_AutoML_2_20250404_153417_model_3,2.18449,4.77199,1.68619,0.0606668,4.77199
GBM_5_AutoML_2_20250404_153417,2.31505,5.35946,1.76653,0.0646864,5.35946
DRF_1_AutoML_2_20250404_153417,2.50636,6.28187,1.93644,0.0704506,6.28187
XRT_1_AutoML_2_20250404_153417,2.75007,7.56287,2.15187,0.0776284,7.56287


In [26]:
pods_pred = aml.leader.predict(pods_test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [32]:
pods_test.head()

team1,team2,days,sessions,year,location,popularity_team1,popularity_team2,pods_required
CSK,GT,Thursday,Evening,2021,Bangalore,9,8,37
DC,GT,Thursday,Evening,2021,Delhi,6,8,30
LSG,RR,Monday,Afternoon,2023,Ahmedabad,7,6,28
RCB,RR,Monday,Evening,2023,Chandigarh,8,6,33
LSG,MI,Sunday,Afternoon,2022,Chennai,7,10,42
KKR,GT,Friday,Evening,2021,Delhi,7,8,33
GT,DC,Wednesday,Evening,2024,Lucknow,8,6,35
KKR,CSK,Thursday,Afternoon,2021,Kolkata,7,9,32
DC,LSG,Wednesday,Evening,2022,Delhi,6,7,30
RCB,GT,Wednesday,Evening,2022,Jaipur,8,8,36


In [31]:
pods_pred.head()

predict
38.4802
30.3355
29.6605
33.0145
39.9625
33.875
33.5146
34.7379
28.5269
36.3392


In [28]:
aml.leader.model_performance(pods_test)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 2.344553093478219
RMSE: 1.531193356006425
MAE: 1.2627683278549142
RMSLE: 0.04249294941242928
Mean Residual Deviance: 2.344553093478219

In [29]:
r2 = aml.leader.model_performance(pods_test).r2()

In [30]:
print(f"R² Score: {r2:.2%}")

R² Score: 92.19%


In [43]:
pods_test_X = pods_test.drop(["pods_required"])

In [44]:
# Predict using only independent features
pods_pred = aml.leader.predict(pods_test_X)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [60]:
# Predict using only independent features
pods_pred.head()

predict
38.4802
30.3355
29.6605
33.0145
39.9625
33.875
33.5146
34.7379
28.5269
36.3392


In [61]:
# Predict using only independent features
pods_test.head()

team1,team2,days,sessions,year,location,popularity_team1,popularity_team2,pods_required
CSK,GT,Thursday,Evening,2021,Bangalore,9,8,37
DC,GT,Thursday,Evening,2021,Delhi,6,8,30
LSG,RR,Monday,Afternoon,2023,Ahmedabad,7,6,28
RCB,RR,Monday,Evening,2023,Chandigarh,8,6,33
LSG,MI,Sunday,Afternoon,2022,Chennai,7,10,42
KKR,GT,Friday,Evening,2021,Delhi,7,8,33
GT,DC,Wednesday,Evening,2024,Lucknow,8,6,35
KKR,CSK,Thursday,Afternoon,2021,Kolkata,7,9,32
DC,LSG,Wednesday,Evening,2022,Delhi,6,7,30
RCB,GT,Wednesday,Evening,2022,Jaipur,8,8,36


In [33]:
current_data = h2o.import_file('/Users/deepakagrawal/AI Hackathon/ipl_test_data.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [35]:
h2o_current = h2o.H2OFrame(current_data)

In [39]:
h2o_current.types

{'team1': 'enum',
 'team2': 'enum',
 'days': 'enum',
 'sessions': 'enum',
 'year': 'int',
 'location': 'enum',
 'popularity_team1': 'int',
 'popularity_team2': 'int',
 'pods_required': 'int'}

In [40]:
h2o_current.head()

team1,team2,days,sessions,year,location,popularity_team1,popularity_team2,pods_required
RCB,RR,Thursday,Afternoon,2025,Jaipur,8,6,33
MI,SRH,Thursday,Evening,2025,Chennai,10,7,44
PBKS,KKR,Tuesday,Afternoon,2025,Bangalore,5,7,28
DC,LSG,Friday,Evening,2025,Chennai,6,7,34
RR,SRH,Friday,Evening,2025,Lucknow,6,7,34
DC,KKR,Tuesday,Evening,2025,Jaipur,6,7,34
RR,KKR,Tuesday,Evening,2025,Mumbai,6,7,34
PBKS,SRH,Wednesday,Evening,2025,Lucknow,5,7,31
MI,RR,Friday,Afternoon,2025,Bangalore,10,6,38
PBKS,LSG,Saturday,Evening,2025,Kolkata,5,7,34


In [46]:
h2o_current_without_pods = h2o_current.drop(["pods_required"])

In [47]:
h2o_current_without_pods.head()

team1,team2,days,sessions,year,location,popularity_team1,popularity_team2
RCB,RR,Thursday,Afternoon,2025,Jaipur,8,6
MI,SRH,Thursday,Evening,2025,Chennai,10,7
PBKS,KKR,Tuesday,Afternoon,2025,Bangalore,5,7
DC,LSG,Friday,Evening,2025,Chennai,6,7
RR,SRH,Friday,Evening,2025,Lucknow,6,7
DC,KKR,Tuesday,Evening,2025,Jaipur,6,7
RR,KKR,Tuesday,Evening,2025,Mumbai,6,7
PBKS,SRH,Wednesday,Evening,2025,Lucknow,5,7
MI,RR,Friday,Afternoon,2025,Bangalore,10,6
PBKS,LSG,Saturday,Evening,2025,Kolkata,5,7


In [53]:
current_pods_pred = aml.leader.predict(h2o_current)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [58]:
current_pods_pred.head()

predict
32.6288
40.5873
29.5339
31.7667
32.5352
33.7843
32.7864
30.5237
35.8444
36.1823


In [59]:
h2o_current.head()

team1,team2,days,sessions,year,location,popularity_team1,popularity_team2,pods_required
RCB,RR,Thursday,Afternoon,2025,Jaipur,8,6,33
MI,SRH,Thursday,Evening,2025,Chennai,10,7,44
PBKS,KKR,Tuesday,Afternoon,2025,Bangalore,5,7,28
DC,LSG,Friday,Evening,2025,Chennai,6,7,34
RR,SRH,Friday,Evening,2025,Lucknow,6,7,34
DC,KKR,Tuesday,Evening,2025,Jaipur,6,7,34
RR,KKR,Tuesday,Evening,2025,Mumbai,6,7,34
PBKS,SRH,Wednesday,Evening,2025,Lucknow,5,7,31
MI,RR,Friday,Afternoon,2025,Bangalore,10,6,38
PBKS,LSG,Saturday,Evening,2025,Kolkata,5,7,34


In [55]:
aml.leader.model_performance(h2o_current)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 6.721207711493824
RMSE: 2.5925292113096474
MAE: 2.1446828492131704
RMSLE: 0.0645509331126285
Mean Residual Deviance: 6.721207711493824

In [56]:
r2_current = aml.leader.model_performance(h2o_current).r2()
print(f"R² Score: {r2_current:.2%}")

R² Score: 75.00%
